In [ ]:

import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
from datetime import datetime, timedelta
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests


sensebox_id = "67cac102d2a4eb00071d6ac9" 

In [ ]:
def get_all_locations(sensebox_id):
    url = f"https://api.opensensemap.org/boxes/{sensebox_id}/locations"
    params = {
        "from-date": "2015-01-01T00:00:00Z",
        "format": "json"
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        raise Exception(f"Fehler beim Abrufen der Daten: {response.status_code}")

    data = response.json()

    locations = []
    for entry in data:
        timestamp = entry.get("timestamp")
        coordinates = entry.get("coordinates")
        if timestamp and coordinates:
            locations.append((timestamp, coordinates))

    return locations

orte = get_all_locations(sensebox_id)

[('2025-03-07T09:48:50.961Z', [104.994063, 77.585865, 2e+31]),
 ('2025-03-07T09:49:38.430Z', [104.994063, 77.585865, 0])]

In [6]:
def show_colored_map(locations):
    if not locations:
        print("Keine Koordinaten vorhanden.")
        return

    # Sortieren nach Zeit
    locations_sorted = sorted(locations, key=lambda x: x[0])
    timestamps = [datetime.fromisoformat(t[0].replace("Z", "+00:00")) for t in locations_sorted]

    # Farbskala vorbereiten
    norm = colors.Normalize(vmin=0, vmax=len(timestamps) - 1)
    colormap = cm.get_cmap("plasma")
    colors_hex = [colors.to_hex(colormap(norm(i))) for i in range(len(timestamps))]

    # Karte auf ersten Punkt zentrieren
    first_coord = locations_sorted[0][1]
    lat = first_coord[1]
    lon = first_coord[0]
    fmap = folium.Map(location=[lat, lon], zoom_start=4)
    marker_cluster = MarkerCluster().add_to(fmap)

    # Marker hinzufügen
    for (timestamp, coord), color in zip(locations_sorted, colors_hex):
        lon = coord[0]
        lat = coord[1]
        alt = coord[2] if len(coord) > 2 else None
        popup_text = f"{timestamp}"
        if alt is not None:
            popup_text += f"<br>Höhe: {alt:.2f} m"

        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            popup=popup_text,
            color=color,
            fill=True,
            fill_color=color
        ).add_to(marker_cluster)

    return fmap  # ← Jupyter zeigt dies automatisch

map = show_colored_map(orte)
orte

C:\Users\tobia\AppData\Local\Temp\ipykernel_14676\970631999.py:12: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap("plasma")


[('2025-03-07T09:48:50.961Z', [104.994063, 77.585865, 2e+31]),
 ('2025-03-07T09:49:38.430Z', [104.994063, 77.585865, 0])]

In [ ]:


def download_measurements_to_dataframe(sensebox_id, from_date=None):
    if from_date is None:
        from_date = datetime(2000, 1, 1)
    elif isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)

    to_date = datetime.now()
    box_data = _get_box_data(sensebox_id)
    sensors = box_data.get("sensors", [])

    print(f"📦 Starte parallelen Download von {len(sensors)} Sensoren...")

    # Dicts zur Zwischenspeicherung
    sensor_dfs = {}
    sensor_timestamps = {}

    # Schritt 1: Daten parallel laden
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {
            executor.submit(_get_sensor_measurements, sensebox_id, sensor, from_date, to_date): sensor
            for sensor in sensors
        }

        for future in as_completed(futures):
            sensor = futures[future]
            try:
                sensor_df, timestamps = future.result()
                if not sensor_df.empty:
                    sensor_dfs[sensor["title"]] = sensor_df
                    sensor_timestamps[sensor["title"]] = timestamps
                    print(f"✅ Sensor '{sensor['title']}' geladen ({len(sensor_df)} Einträge)")
                else:
                    print(f"⚠️ Sensor '{sensor['title']}' enthält keine Daten")
            except Exception as e:
                print(f"❌ Fehler bei Sensor '{sensor['title']}': {e}")

    if not sensor_dfs:
        print("❌ Keine Daten geladen.")
        return pd.DataFrame(), None, {}

    # Schritt 2: DataFrames zusammenführen
    combined_df = pd.concat(sensor_dfs.values(), axis=1, join='outer')
    combined_df = combined_df.sort_index()
    combined_df = combined_df[~combined_df.index.duplicated(keep='first')]
    last_timestamp = combined_df.index.max().isoformat()

    return combined_df, last_timestamp, sensor_timestamps


def _get_box_data(sensebox_id):
    url = f"https://api.opensensemap.org/boxes/{sensebox_id}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()


def _get_sensor_measurements(sensebox_id, sensor, from_date, to_date):
    sensor_id = sensor["_id"]
    sensor_title = sensor["title"]
    step = timedelta(days=30)
    current_start = from_date
    sensor_df = pd.DataFrame()

    while current_start < to_date:
        current_end = min(current_start + step, to_date)
        data = _fetch_measurement_chunk(sensebox_id, sensor_id, current_start, current_end)

        if len(data) == 10000 and step > timedelta(days=1):
            step = timedelta(days=max(step.days // 2, 1))
            continue

        if data:
            batch_df = _parse_measurements_to_df(data, sensor_title)
            sensor_df = pd.concat([sensor_df, batch_df], ignore_index=True)

        current_start = current_end
        step = timedelta(days=30)

    if not sensor_df.empty:
        sensor_df = sensor_df.set_index("createdAt")
        sensor_df = sensor_df[~sensor_df.index.duplicated(keep='first')]
        sensor_df = sensor_df.sort_index()

    return sensor_df, sensor_df.index.to_list() if not sensor_df.empty else []


def _fetch_measurement_chunk(sensebox_id, sensor_id, from_dt, to_dt):
    url = f"https://api.opensensemap.org/boxes/{sensebox_id}/data/{sensor_id}"
    params = {
        "format": "json",
        "from-date": from_dt.isoformat() + "Z",
        "to-date": to_dt.isoformat() + "Z"
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"❌ Fehler bei Datenabruf: {from_dt} – {to_dt}")
        return []
    return response.json()


def _parse_measurements_to_df(data, column_name):
    records = []
    for entry in data:
        try:
            records.append({
                "createdAt": pd.to_datetime(entry["createdAt"]),
                column_name: float(entry["value"])
            })
        except Exception:
            continue
    return pd.DataFrame(records)

In [21]:
# Daten herunterladen (ab einem beliebigen Startdatum)
combined_df, last_timestamp, sensor_timestamps = download_measurements_to_dataframe(sensebox_id, from_date="2024-01-01")

# 1. Zeitstempel pro Sensor anzeigen
print("\n📅 Zeitstempel pro Sensor:")
for sensor_name, timestamps in sensor_timestamps.items():
    print(f"{sensor_name}: {len(timestamps)} Messungen (von {timestamps[0]} bis {timestamps[-1]})")


📦 Starte parallelen Download von 15 Sensoren...
⚠️ Sensor 'VOC' enthält keine Daten
✅ Sensor 'Luftfeuchte SHT31' geladen (313 Einträge)
✅ Sensor 'Temperatur SHT31' geladen (313 Einträge)
✅ Sensor 'Bodentemperatur Port 2 20 cm West' geladen (17276 Einträge)
✅ Sensor 'Luftfeuchte BME680' geladen (17499 Einträge)
✅ Sensor 'Bodenfeuchte Port 2 20 cm West' geladen (17276 Einträge)
✅ Sensor 'Bodenfeuchte Port 1 10 cm West' geladen (17276 Einträge)
✅ Sensor 'atm. Luftdruck BME680' geladen (17499 Einträge)
✅ Sensor 'Bodentemperatur Port 1 10 cm West' geladen (17276 Einträge)
✅ Sensor 'Lufttemperatur BME680' geladen (17499 Einträge)
✅ Sensor 'Bodentemperatur Port 3 10 cm Ost' geladen (17186 Einträge)
✅ Sensor 'Bodenfeuchte Port 3 10 cm Ost' geladen (17186 Einträge)
✅ Sensor 'Bodenfeuchte Port 5 20 cm Ost' geladen (17186 Einträge)
✅ Sensor 'Heatflux (Osten) 10 cm' geladen (17186 Einträge)
✅ Sensor 'Bodentemperatur Port 5 20 cm Ost' geladen (17186 Einträge)

📅 Zeitstempel pro Sensor:
Luftfeuchte 

In [23]:
combined_df

,Luftfeuchte SHT31,Temperatur SHT31,Bodentemperatur Port 2 20 cm West,Luftfeuchte BME680,Bodenfeuchte Port 2 20 cm West,Bodenfeuchte Port 1 10 cm West,atm. Luftdruck BME680,Bodentemperatur Port 1 10 cm West,Lufttemperatur BME680,Bodentemperatur Port 3 10 cm Ost,Bodenfeuchte Port 3 10 cm Ost,Bodenfeuchte Port 5 20 cm Ost,Heatflux (Osten) 10 cm,Bodentemperatur Port 5 20 cm Ost
createdAt,,,,,,,,,,,,,,
2025-03-07 10:16:33.500000+00:00,34.9050,22.4739,NaN,27.9473,NaN,NaN,959.67,NaN,24.8956,NaN,NaN,NaN,NaN,NaN
2025-03-07 10:17:35.822000+00:00,34.7967,22.4445,NaN,27.8422,NaN,NaN,959.67,NaN,24.8276,NaN,NaN,NaN,NaN,NaN
2025-03-07 10:18:47.413000+00:00,34.9920,22.4178,NaN,28.0790,NaN,NaN,959.69,NaN,26.2907,NaN,NaN,NaN,NaN,NaN
2025-03-07 10:19:49.770000+00:00,35.2255,22.4739,NaN,58.8659,NaN,NaN,959.72,NaN,25.6908,NaN,NaN,NaN,NaN,NaN
2025-03-07 10:21:00.211000+00:00,56.6522,20.6313,NaN,30.8215,NaN,NaN,959.70,NaN,25.1217,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-16 06:30:08.997000+00:00,NaN,NaN,11.2300,76.1345,22.3700,28.6100,958.40,10.6600,12.6662,10.5150,17.5300,18.8700,-7.7882,11.1450
2025-05-16 06:31:08.239000+00:00,NaN,NaN,11.2067,76.0358,22.3700,28.6100,958.40,10.6633,12.7096,10.5167,17.5300,18.8700,-7.7882,11.1267
2025-05-16 06:32:26.954000+00:00,NaN,NaN,11.2475,75.1191,22.3450,28.6650,958.38,10.6650,12.7547,10.5150,17.5375,18.8500,-7.7882,11.1325


In [24]:
# als CSV-Datei speichern
combined_df.to_csv("sensordaten.csv", index=True)


In [25]:

# Zeitpunkt des letzten Werts speichern
last_time = df.index.max().isoformat()  
with open("last_measurement.txt", "w") as f:
    f.write(last_time)



In [26]:
# Letzten Zeitpunkt der Datei einlesen
def get_last_timestep():
    with open("last_measurement.txt", "r") as f:
        last_time_str = f.read().strip()
        f.close()
    # In datetime-Objekt umwandeln
    return datetime.fromisoformat(last_time_str)
